In [12]:
from numpy import *
def loadDataset():
    postingList = [['my','dog','has','flea','problems','help','please'],
                   ['maybe','not','take','him','to','dog','park','stupid'],
                   ['my','dalmation','is','so','cute','I','love','him'],
                   ['stop','posting','stupid','worthless','garbage'],
                   ['mr','licks','ate','my','steak','how','to','stop','him'],
                   ['quit','buying','worthless','dog','food','stupid']
    ]
    classVec = [0,1,0,1,0,1]
    return postingList,classVec

def createVocabList(dataset):
    vocabSet = set()
    for document in dataset:
        vocabSet = vocabSet | set(document)
    return list(vocabSet)

def setOfWords2Vec(vocabList,inputSet):
    returnVec = [0]*len(vocabList) 
    for word in inputSet:
        if word in inputSet:
            returnVec[vocabList.index(word)]=1
        else:
            print ("the word: %s is not in the vocabulary" % word)
    return returnVec

    

In [7]:
listPosts,listClasses = loadDataset()
myVocabList = createVocabList(listPosts)
myVocabList

['stupid',
 'flea',
 'ate',
 'food',
 'garbage',
 'please',
 'steak',
 'problems',
 'how',
 'mr',
 'to',
 'dalmation',
 'posting',
 'is',
 'dog',
 'maybe',
 'love',
 'has',
 'so',
 'my',
 'park',
 'cute',
 'him',
 'stop',
 'buying',
 'licks',
 'worthless',
 'I',
 'take',
 'not',
 'quit',
 'help']

In [11]:
setOfWords2Vec(myVocabList,listPosts[0])

[0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1]

In [20]:
#朴素贝叶斯分类器 p(c|w)=(p(w|c1)*p(w|c2)*p(w|c3)..)*p(c)/p(w)
def trainNB0(trainMatrix,trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numTrainDocs)#先验概率
    p0Num = ones(numWords); p1Num = ones(numWords)
    p0Denom =2.0; p1Denom =2.0 #将词出现概率初始化为0.5，避免p(w|c1)=0的情况
    for i in range(numTrainDocs):
        if trainCategory[i]==1:
            p1Num +=trainMatrix[i]
            p1Denom +=sum(trainMatrix[i])
        else:
            p0Num +=trainMatrix[i]
            p0Denom +=sum(trainMatrix[i])
    p1Vec = log(p1Num/p1Denom)
    p0Vec = log(p0Num/p0Denom) #在给定文档类别条件下词汇表中单词的出现概率
    return p0Vec,p1Vec,pAbusive


In [21]:
trainMat=[]
for post in listPosts:
    trainMat.append(setOfWords2Vec(myVocabList,post))
trainNB0(trainMat,listClasses)

(array([-3.25809654, -2.56494936, -2.56494936, -3.25809654, -3.25809654,
        -2.56494936, -2.56494936, -2.56494936, -2.56494936, -2.56494936,
        -2.56494936, -2.56494936, -3.25809654, -2.56494936, -2.56494936,
        -3.25809654, -2.56494936, -2.56494936, -2.56494936, -1.87180218,
        -3.25809654, -2.56494936, -2.15948425, -2.56494936, -3.25809654,
        -2.56494936, -3.25809654, -2.56494936, -3.25809654, -3.25809654,
        -3.25809654, -2.56494936]),
 array([-1.65822808, -3.04452244, -3.04452244, -2.35137526, -2.35137526,
        -3.04452244, -3.04452244, -3.04452244, -3.04452244, -3.04452244,
        -2.35137526, -3.04452244, -2.35137526, -3.04452244, -1.94591015,
        -2.35137526, -3.04452244, -3.04452244, -3.04452244, -3.04452244,
        -2.35137526, -3.04452244, -2.35137526, -2.35137526, -2.35137526,
        -3.04452244, -1.94591015, -3.04452244, -2.35137526, -2.35137526,
        -2.35137526, -3.04452244]),
 0.5)

In [24]:
log(array([1,2,3])/2)

array([-0.69314718,  0.        ,  0.40546511])

In [28]:
def classifyNB(vec2Classify,p0Vec,p1Vec,pClass1):
    p1 = sum(vec2Classify*p1Vec) + log(pClass1)
    p0 = sum(vec2Classify*p0Vec) + log(1.0-pClass1)
    if p1>p0:
        return 1
    else:
        return 0
    
def testingNB():
    listPosts,listClasses = loadDataset()
    myVocabList = createVocabList(listPosts)
    trainMat =[]
    for post in listPosts:
        trainMat.append(setOfWords2Vec(myVocabList,post))
    p0V,p1V,pAb = trainNB0(array(trainMat),array(listClasses))
    testEntry = ['love','my','dalmation']
    thisDoc = array(setOfWords2Vec(myVocabList,testEntry))
    print (testEntry,'classified as:',classifyNB(thisDoc,p0V,p1V,pAb))
    testEntry = ['stupid','garbage']
    thisDoc = array(setOfWords2Vec(myVocabList,testEntry))
    print (testEntry,'classified as:',classifyNB(thisDoc,p0V,p1V,pAb))

In [29]:
testingNB()

['love', 'my', 'dalmation'] classified as: 0
['stupid', 'garbage'] classified as: 1


In [ ]:
#朴素贝叶斯词袋模型
def bagOfWords2VecMN(vocabList,inputSet):
    returnVec = [0]*len(vocabList) 
    for word in inputSet:
        if word in inputSet:
            returnVec[vocabList.index(word)]+=1
        else:
            print ("the word: %s is not in the vocabulary" % word)
    return returnVec

In [32]:
int(random.uniform(0,9))

2